In [1]:
import boto3
import csv
import json
import os
import requests
from bs4 import BeautifulSoup
from io import BytesIO;
from os import listdir
from os.path import isfile, join
from scrapy.http import TextResponse

In [2]:
path = "/home/propietari/Documents/fitxers importants/WRI/Scraping_results/"

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]


In [3]:
onlyfiles

['ElSalvador_20210730.csv',
 'USFR_2_20210311.csv',
 'USFR_20210311.csv',
 'USFR_1_20210311.csv',
 'USFR_20210702.csv',
 'LeyChile_20210702.csv',
 'Mexico_20210808.csv',
 'India_20210310.csv',
 'Mexico_20210703.csv',
 'USFR_20210310.csv',
 'India_20210703.csv',
 'Mexico_20210809.csv',
 'El_Salvador_no_duplicates.csv']

In [4]:
# path = 'C:/Users/jordi/Documents/claus/'
key_path = '/home/propietari/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = key_path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)


### El Salvador

In [18]:
# Loading the csv files from scraping to be compared
file_position = 0
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    ElSalvador_20210730 = [row[3] for row in reader]

file_position = 12
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    El_Salvador_no_duplicates = [row[7] for row in reader]

In [53]:
ElSalvador_20210730_SET = set(ElSalvador_20210730)
print(f"There are {len(El_Salvador_no_duplicates)} items in the old file and {len(ElSalvador_20210730)} in the new one")
print(f"There are {len(ElSalvador_20210730) - len(ElSalvador_20210730_SET)} duplicates in the new version")
print(f"The difference between the first scraping and the last version is of {len(El_Salvador_no_duplicates) - len(ElSalvador_20210730)} entries")
not_in_newest_version = set(El_Salvador_no_duplicates) - ElSalvador_20210730_SET
print(f"There are {len(not_in_newest_version)} files in the old version that are not in the new version")
not_in_oldest_version =  ElSalvador_20210730_SET - set(El_Salvador_no_duplicates)
print(f"There are {len(not_in_oldest_version)} files in the new version that are not in the old version")
print(f"There are {len(set(El_Salvador_no_duplicates) | ElSalvador_20210730_SET)} unique ids in the union of the files")

There are 460 items in the old file and 271 in the new one
There are 0 duplicates in the new version
The difference between the first scraping and the last version is of 189 entries
There are 236 files in the old version that are not in the new version
There are 47 files in the new version that are not in the old version
There are 507 unique ids in the union of the files


In [58]:
# Going to merge the two files in one. The fields were not saved in the sae order so it is necessary to rearrange them

file_position = 0
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    ElSalvador_20210730 = list(reader)

file_position = 12
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    El_Salvador_no_duplicates = list(reader)
    
final_list = []
sv = []
ids = {}
for item in El_Salvador_no_duplicates:
    if item[7] not in ids:
        final_item = []
        final_item.append(item[7])
        final_item.append('El Salvador')
        final_item.append('Diario Oficial')
        final_item.append(item[1])
        final_item.append(item[4])
        final_item.append(item[2])
        final_item.append(item[6])
        final_item.append(item[5])
        final_item.append(item[3])
        final_list.append(final_item)
        sv.append(item[7])
    
for item in ElSalvador_20210730:
    if item[3] not in ids:
        final_item = []
        final_item.append(item[3])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[7])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3]] = 0
        sv.append(item[3])

In [61]:
sv[0]

'765d4ea2383aaf45bb29bf75feadea6e06a80fd2'

In [81]:
# Checking the differences with bucket files

language = "spanish"
in_prefix = f"{language}_documents/HSSC/sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0
        
processed_list = []
for key in ids_S3:
    processed_list.append(key)

In [75]:
print(f"The number of scraped items that are not in sentences is {len(set(sv) - set(processed_list))}")
print(f"The number of scraped items that are not in sentences is {len(set(sv) - set(new_list))}")
print(len(set(sv)))

The number of scraped items that are not in sentences is 236
The number of scraped items that are not in sentences is 271
507


In [98]:
# Here we go and fetch the missing files and put them in the raw_pdf_updated folder
prefix = f"{language}_documents/raw_pdf_updated/"


counter = 0
bucket = s3.Bucket(s3BucketName)
for item in final_list:
    if item[0] not in ids_S3:
        counter += 1
        key = prefix + item[0] + ".pdf"
        with requests.get(item[8], stream=True) as r:
            bucket.upload_fileobj(r.raw, key)
print(counter)

236


### Chile

In [62]:
file_position = 5
with open(path + onlyfiles[file_position], 'r', encoding="ISO-8859-1") as f:#, encoding = "Latin1"
    reader = csv.reader(f)
    chile = list(reader)
    
# final_list = []
# ids = {}
# dates = {}
ch = []
for item in chile:
    if item[3].split(".")[0] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
#         dates[item[6]] = 0
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0
        ch.append(item[3].split(".")[0])


In [76]:
print(f"The number of scraped items that are not in sentences is {len(set(ch) - set(processed_list))}")
print(f"The number of scraped items that are not in sentences is {len(set(ch) - set(new_list))}")
print(len(set(ch)))

The number of scraped items that are not in sentences is 4
The number of scraped items that are not in sentences is 856
860


In [196]:
# Checking the differences with bucket files

language = "spanish"
in_prefix = f"{language}_documents/HSSC/sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0

In [158]:
print(f"The number of scraped items that are not in sentences is {len(ids.keys() - ids_S3.keys())}")

The number of scraped items that are not in sentences is 4


In [167]:
# Here we go and fetch the missing files and put them in the raw_pdf_updated folder
prefix = f"{language}_documents/text_files/HSSC/new/"
    
counter = 0
for item in final_list:
    if item[0] not in ids_S3:
        counter += 1
        key = prefix + item[0] + ".txt"
        with requests.get(item[8], stream=True) as r:
            s3.Object(s3BucketName, key).put(Body=r.content)

print(counter)

4


### Mexico

In [83]:
file_position = 6
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:#, encoding="ISO-8859-1"
    reader = csv.reader(f)
    mexico_august = list(reader)
    
# final_list = []
ids = {}
# dates = {}
mx_au = []
for item in mexico_august:
    if item[3].split(".")[0] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
#         dates[item[6]] = 0
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0
        mx_au.append(item[3].split(".")[0])
        
file_position = 8
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:#, encoding="ISO-8859-1"
    reader = csv.reader(f)
    mexico_july = list(reader)
    
mx_ju = []
for item in mexico_july:
    if item[3].split(".")[0] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
#         dates[item[6]] = 0
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0
        mx_ju.append(item[3].split(".")[0])


In [85]:
print(f"The number of newly scraped items that are not processed is {len(set(mx_au) - set(processed_list))}")
print(f"The number of scraped items that are not in sentences is {len(set(mx_au) - set(new_list))}")
print(len(set(mx_au)))
print(f"The number of newly scraped items that are not processed is {len(set(mx_ju) - set(processed_list))}")
print(f"The number of scraped items that are not in sentences is {len(set(mx_ju) - set(new_list))}")
print(len(set(mx_ju)))
print(f"The number of scraped items that are not in sentences is {len(set(mx_ju) - set(mx_au))}")

The number of newly scraped items that are not processed is 3956
The number of scraped items that are not in sentences is 1140
5096
The number of newly scraped items that are not processed is 20
The number of scraped items that are not in sentences is 3946
3948
The number of scraped items that are not in sentences is 3948


In [64]:
# Checking the differences with bucket files

language = "spanish"
in_prefix = f"{language}_documents/HSSC/sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0

processed_list = []
for key in ids_S3:
    processed_list.append(key)
        
in_prefix = f"{language}_documents/text_files/HSSC/new/"        
ids_S3_new = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".txt" in obj.key:
        name = obj.key.split("/")[-1].split("_")[0].split(".")[0]
        ids_S3_new[name] = 0
        
new_list = []
for key in ids_S3_new:
    new_list.append(key)

In [69]:
print(f"The number of scraped new that are not in processed is {len(set(new_list) - set(processed_list))}")
print(f"The number of scraped processed that are not in new is {len(set(processed_list) - set(new_list))}")
print(processed_list[0])
print(new_list[0])


The number of scraped new that are not in processed is 4351
The number of scraped processed that are not in new is 8562
00009282cf7af3ed1a9e25e1b43791837e7c144a
000fe0bdf5e9a0024ca7e9bb8b15dadf5648ae8c


In [32]:
# Here we go and fetch the missing files and put them in the raw_pdf_updated folder
language = "spanish"
prefix = f"{language}_documents/text_files/HSSC/new/"


def save_to_s3(s3, bucket, file_key, text):
    s3.Object(bucket, file_key).put(Body = text)

def parse_other(response, key):
    soup = BeautifulSoup(response.css('div#DivDetalleNota').get(), features = "lxml")
    paragraphs = soup.find_all("p")
    text = ""
    if len(paragraphs) == 0:
        text = text + soup.text
    else:
        tables = soup.find_all("td")
        for line in paragraphs[1:]:
            text = text + line.text + "\n"
        text = text + "<table>" + "\n"
        for cell in tables:
            if "En el documento que usted está visualizando" not in cell.text:
                text = text + cell.text + "\n"
        text = text + "<\\table>" + "\n"
    save_to_s3(s3, s3BucketName, key, text.replace("\t", ""))  


counter = 0
for item in final_list:
    if item[0] not in ids_S3:
        counter += 1
        key = prefix + item[0] + ".txt"
        with requests.get(item[8], stream=True) as r:
            resp = TextResponse(body=r.content, url=item[8])
            parse_other(resp, key)
        print(counter, end='\r')

### India

In [102]:
# Loading the csv files from scraping to be compared
file_position = 6
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    india_old = [row[3] for row in reader]

file_position = 9
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    india_new = [row[7] for row in reader]

In [119]:
file_position = 6
with open(path + onlyfiles[file_position], 'r', encoding="ISO-8859-1") as f:#, encoding = "Latin1"
    reader = csv.reader(f)
    india_old = list(reader)

file_position = 9
with open(path + onlyfiles[file_position], 'r', encoding="ISO-8859-1") as f:
    reader = csv.reader(f)
    india_new = list(reader)

In [120]:
india_old[0]

['/ Department of AGRICULTURE AND MARKETING',
 'India',
 'India Code',
 '859644d69fdd0b1e66f1dcf5a42f2d9a3f31fe90.pdf',
 'https://www.indiacode.nic.in/bitstream/123456789/6515/1/kishan_ayog_2016_hindi_.pdf#search=Miner OR Ore OR Pit OR Bog OR Buffer OR Corridor OR (Country planning) OR Cropland OR (Degraded land) OR Desert OR Floodplain OR Forestland OR Freshwater OR Grassland OR (Land use) OR Landowner OR Mangrove OR Marsh OR Meadow [1950 TO 2021]',
 'Act',
 ' 2016-11-30',
 ' 201629',
 ' Uttarakhand',
 '',
 'The Uttarakhand state farmer comission Act',
 'https://www.indiacode.nic.in/handle/123456789/2109?view_type=search&sam_handle=123456789/1362']

In [17]:
india_new[0]

NameError: name 'india_new' is not defined